# Regression Week 2: Multiple Linear Regression Quiz 2

Estimating Multiple Regression Coefficients (Gradient Descent)

In the first notebook we explored multiple regression using GraphLab Create. Now we will use SFrames along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Part 0 - Load the data

In [2]:
dtype_dict = {
    'bathrooms':float, 
    'waterfront':int, 
    'sqft_above':int, 
    'sqft_living15':float, 
    'grade':int, 
    'yr_renovated':int, 
    'price':float, 
    'bedrooms':float, 
    'zipcode':str,
    'long':float,
    'sqft_lot15':float, 
    'sqft_living':float, 
    'floors':str,
    'condition':int,
    'lat':float,
    'date':str,
    'sqft_basement':int, 
    'yr_built':int,
    'id':str,
    'sqft_lot':int,
    'view':int
}

train_data = pd.read_csv('../kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('../kc_house_test_data.csv', dtype=dtype_dict)

### Part 1 - features to numpy

write a function that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function should return a features_matrix (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It should also return an output_array which is an array of the values of the output in the data set (e.g. ‘price’).

In [3]:
def get_numpy_data(data_frame, features, output):

    features_matrix = np.array([np.ones(len(data_frame))] + [data_frame[feature] for feature in features])

    output_array = np.array(data_frame[output])
    
    return features_matrix, output_array

### Part 2 - predict

 If the features matrix (including a column of 1s for the constant) is stored as a 2D array (or matrix) and the regression weights are stored as a 1D array then the predicted output is just the dot product between the features matrix and the weights (with the weights on the right). Write a function ‘predict_output’ which accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and returns a 1D array ‘predictions’.

In [4]:
def predict_outcome(feature_matrix, weights):
    return feature_matrix.T.dot(weights)

### Part 3 - derive

If we have a the values of a single input feature in an array ‘feature’ and the prediction ‘errors’ (predictions - output) then the derivative of the regression cost function with respect to the weight of ‘feature’ is just twice the dot product between ‘feature’ and ‘errors’. Write a function that accepts a ‘feature’ array and ‘error’ array and returns the ‘derivative’ (a single number)

In [5]:
def feature_derivative(errors, feature):
    return 2 * feature.dot(errors)

### Part 4 - descend

Now we will use our predict_output and feature_derivative to write a gradient descent function. Although we can compute the derivative for all the features simultaneously (the gradient) we will explicitly loop over the features individually for simplicity. Write a gradient descent function that does the following:

- Accepts a numpy feature_matrix 2D array, a 1D output array, an array of initial weights, a step size and a convergence tolerance.
- While not converged updates each feature weight by subtracting the step size times the derivative for that feature given the current weights
- At each step computes the magnitude/length of the gradient (square root of the sum of squared components)
- When the magnitude of the gradient is smaller than the input tolerance returns the final weight vector.

In [6]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = predict_outcome(feature_matrix, weights)
        
        # compute the errors as predictions - output:
        errors = predictions - output
        
        gradient_sum_squares = 0 # initialize the gradient
        
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix.T[:,i])

            # add the squared derivative to the gradient magnitude
            gradient_sum_squares = derivative ** 2
            
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - step_size * derivative
            
        gradient_magnitude = np.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

### Part 5

Now we will run the regression_gradient_descent function on some actual data. In particular we will use the gradient descent to estimate the model from Week 1 using just an intercept and slope. Use the following parameters:

- features: ‘sqft_living’
- output: ‘price’
- initial weights: -47000, 1 (intercept, sqft_living respectively)
- step_size = 7e-12
- tolerance = 2.5e7

In [7]:
simple_features = ['sqft_living']
my_output= 'price'
simple_feature_matrix, output = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [8]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,                                             tolerance)

### Part 6 

What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?



In [9]:
simple_weights[1]

281.91211917520917

### Part 7

Now build a corresponding ‘test_simple_feature_matrix’ and ‘test_output’ using test_data. Using ‘test_simple_feature_matrix’ and ‘simple_weights’ compute the predicted house prices on all the test data.



In [10]:
test_simple_feature_matrix, test_output = get_numpy_data(test_data, simple_features, my_output)
test_predictions = predict_outcome(test_simple_feature_matrix, simple_weights)
test_predictions

array([ 356134.443255  ,  784640.86440132,  435069.83662406, ...,
        663418.65315598,  604217.10812919,  240550.47439317])

### Part 8

Now compute RSS on all test data for this model. Record the value and store it for later



In [11]:
test_rss = np.sum((test_predictions - test_output) ** 2)

### Part 9

Now we will use the gradient descent to fit a model with more than 1 predictor variable (and an intercept). Use the following parameters:

- model features = ‘sqft_living’, ‘sqft_living_15’
- output = ‘price’
- initial weights = [-100000, 1, 1] (intercept, sqft_living, and sqft_living_15 respectively)
- step size = 4e-12
- tolerance = 1e9


In [12]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

my_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size, tolerance)
my_weights

array([ -9.99999688e+04,   2.45035463e+02,   6.53198571e+01])

### Part 10

Use the regression weights from this second model (using sqft_living and sqft_living_15) and predict the outcome of all the house prices on the TEST data.

In [13]:
test_feature_matrix, test_output = get_numpy_data(test_data, model_features, my_output)
test_predictions2 = predict_outcome(test_feature_matrix, my_weights)
test_predictions2

array([ 366670.08864306,  762639.14060956,  386290.12537701, ...,
        682095.43732627,  585567.28873248,  216562.4574978 ])

### Part 11

What is the actual price for the 1st house in the Test data set?

In [14]:
test_output[0]

310000.0

### Part 12

Now compute RSS on all test data for the second model. Record the value and store it for later.

In [15]:
test2_rss = np.sum((test_predictions2 - test_output) ** 2)
test2_rss, test_rss

(270261660020645.31, 275400044902128.31)